##This notebook can be used to classify the images with eddy from non-eddy using google colab platform.

###Mount directories from google drive

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Import libraries

###libraries

In [59]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import keras
import os

In [60]:
data_path = ('/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/demo_png')
data_path = ('/content/drive/MyDrive/path_to_dir')
img_path= data_path
os.chdir(img_path)
print(os.path.abspath(os.getcwd()))

/content/drive/.shortcut-targets-by-id/1KS2muRwn4VXnctrNWa5dx0K8BIIRSELD/ESIP-Ocean-Eddy-Detection/demo_png


###libraries

In [61]:
# import tensorflow as tf
from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

In [62]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)
#https://machinelearningmastery.com/reproducible-results-neural-networks-keras/

In [63]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
# plot_model(model, to_file='model_plot11.png', show_shapes=True, show_layer_names=True)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 60, 60, 32)        18464     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 30, 30, 32)      

In [64]:
batch_size = 16

from keras.preprocessing.image import ImageDataGenerator

# # this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False) ## cannot do flip

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(
        rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        # '/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/demo_png/train',  # this is the target directory
        '/content/drive/MyDrive/path_to_dir/train' # this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        color_mode = 'grayscale',
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        # '/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/demo_png/validation',
        '/content/drive/MyDrive/path_to_dir/validation',
        target_size=(256, 256),
        color_mode = 'grayscale',
        batch_size=1,
        class_mode='binary')

# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(
        # '/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/demo_png/test',
        '/content/drive/MyDrive/path_to_dir/test',
        target_size=(256, 256),
        color_mode = 'grayscale',
        batch_size=1,
        class_mode=None,
        shuffle=False)

Found 42 images belonging to 2 classes.
Found 22 images belonging to 2 classes.
Found 15 images belonging to 2 classes.


In [65]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [66]:
h = model.fit(
        train_generator,
        # steps_per_epoch=200 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        # validation_steps=80 // batch_size
        )

Epoch 1/10
3/3 [==============================] - 5s 1s/step - loss: 0.3944 - accuracy: 0.5952 - val_loss: 0.0793 - val_accuracy: 0.9091
Epoch 2/10
3/3 [==============================] - 4s 1s/step - loss: 0.0452 - accuracy: 0.9524 - val_loss: 0.0816 - val_accuracy: 0.9091
Epoch 3/10
3/3 [==============================] - 4s 1s/step - loss: 0.0408 - accuracy: 0.9524 - val_loss: 0.0667 - val_accuracy: 0.9091
Epoch 4/10
3/3 [==============================] - 4s 1s/step - loss: 0.0335 - accuracy: 0.9762 - val_loss: 0.0581 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 4s 1s/step - loss: 0.0295 - accuracy: 1.0000 - val_loss: 0.0524 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 4s 2s/step - loss: 0.0274 - accuracy: 1.0000 - val_loss: 0.0442 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 4s 1s/step - loss: 0.0209 - accuracy: 1.0000 - val_loss: 0.0342 - val_accuracy: 1.0000
Epoch 8/10
3/3 [=========================

In [67]:
print("Average accuracy: ",sum(h.history['accuracy'])/10)

Average accuracy:  0.9476190447807312
